##### Copyright 2021 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# アーム単位の特徴量を使った多腕バンディットのチュートリアル

### はじめに

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/per_arm_bandits_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/per_arm_bandits_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/per_arm_bandits_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/per_arm_bandits_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>

このチュートリアルは、特徴別（ジャンル、リリース年など）にまとめられた映画のリストなど、アクション（アーム）に独自の特徴量のある文脈的バンディット問題に TF-Agents ライブラリを使用する手順を示すガイドです。

### 前提条件

TF-Agents のバンディットライブラリについてある程度の知識があること、特にこのチュートリアルの前に「[TF-Agents におけるバンディットのチュートリアル](https://github.com/tensorflow/agents/tree/master/docs/tutorials/bandits_tutorial.ipynb)」を実行したことを前提としています。

## アームの特徴量を使用した多腕バンディット

「古典的」な多腕バンディットの設定では、エージェントは時間ステップごとのコンテキストベクトル（観測）を受け取り、累積報酬を最大化するために、番号付きのアクション（アーム）の有限セットから選択する必要があります。

では、エージェントがユーザーに、次に見る英があを推奨するというシナリオについて考えてみましょう。決定を行うたびに、エージェントはコンテキストとしてユーザーに関する情報（鑑賞履歴、好みのジャンルなど）と選択する映画のリストを受け取ります。

コンテキストとしてのユーザー情報を使ってこの問題を定式化すると、アームは `movie_1, movie_2, ..., movie_K` となりますが、このアプローチには以下のような複数の欠点があります。

- アクション数はすべてシステム内の映画に限られ、新しい映画を追加するのは面倒である。
- エージェントは映画ごとのモデルを学習する必要がある。
- 映画同士の類似性が考慮されていない。

映画に番号を付ける代わりに、より直感的に、ジャンル、長さ、キャスト、レーティング、年などを含む一連の特徴量で映画を表現することができます。このアプローチには、以下のようにさまざまなメリットがあります。

- 映画の一般化。
- エージェントは、ユーザーと映画の特徴量で報酬をモデル化する報酬関数を 1 つ学習するだけでよい。
- システム上の映画の追加と削除を簡単に行える。

この新しい設定では、アクション数は時間ステップごとに異なることが可能です。


## TF-Agents におけるアーム単位のバンディット

TF-Agents Bandit スイートは、アーム単位のケースにも使用できるように開発されています。アーム単位の環境が用意されており、ポリシーとエージェントのほとんどもアーム単位のモードで動作可能です。

コーディング例に進む前に、必要なインポートを行いましょう。

### インストール

In [ ]:
!pip install tf-agents

### インポート

In [ ]:
import functools
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.bandits.environments import stationary_stochastic_per_arm_py_environment as p_a_env
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import tf_py_environment
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step as ts

nest = tf.nest

### パラメータ -- 自由にいじってみましょう

In [ ]:
# The dimension of the global features.
GLOBAL_DIM = 40  #@param {type:"integer"}
# The elements of the global feature will be integers in [-GLOBAL_BOUND, GLOBAL_BOUND).
GLOBAL_BOUND = 10  #@param {type:"integer"}
# The dimension of the per-arm features.
PER_ARM_DIM = 50  #@param {type:"integer"}
# The elements of the PER-ARM feature will be integers in [-PER_ARM_BOUND, PER_ARM_BOUND).
PER_ARM_BOUND = 6  #@param {type:"integer"}
# The variance of the Gaussian distribution that generates the rewards.
VARIANCE = 100.0  #@param {type: "number"}
# The elements of the linear reward parameter will be integers in [-PARAM_BOUND, PARAM_BOUND).
PARAM_BOUND = 10  #@param {type: "integer"}

NUM_ACTIONS = 70  #@param {type:"integer"}
BATCH_SIZE = 20  #@param {type:"integer"}

# Parameter for linear reward function acting on the
# concatenation of global and per-arm features.
reward_param = list(np.random.randint(
      -PARAM_BOUND, PARAM_BOUND, [GLOBAL_DIM + PER_ARM_DIM]))

### 単純なアーム単位の環境

他の[チュートリアル](https://github.com/tensorflow/agents/tree/master/docs/tutorials/bandits_tutorial.ipynb)で説明した静止した確定的環境には、アーム単位の相当物があります。

アーム単位の環境を初期化するには、以下を生成する関数を定義する必要があります。

- *グローバル特徴量とアーム単位の特徴量*: これらの関数には入力パラメータがなく、呼び出されると単一（グローバルまたはアーム単位）の特徴量ベクトルを生成します。
- *報酬*: この関数はパラメータとしてグローバルとアーム単位の特徴量ベクトルの連結を取り、報酬を生成します。基本的に、これが、エージェントが「推測」する必要のある関数です。アーム単位のケースでは、報酬関数はすべてのアームで同一であることに注意してください。これが、エージェントがアームごとに個別に報酬関数を推定する必要のある古典的なバンディットのケースとの根本的な違いです。


In [ ]:
def global_context_sampling_fn():
  """This function generates a single global observation vector."""
  return np.random.randint(
      -GLOBAL_BOUND, GLOBAL_BOUND, [GLOBAL_DIM]).astype(np.float32)

def per_arm_context_sampling_fn():
  """"This function generates a single per-arm observation vector."""
  return np.random.randint(
      -PER_ARM_BOUND, PER_ARM_BOUND, [PER_ARM_DIM]).astype(np.float32)

def linear_normal_reward_fn(x):
  """This function generates a reward from the concatenated global and per-arm observations."""
  mu = np.dot(x, reward_param)
  return np.random.normal(mu, VARIANCE)

これで、環境を初期化できるようになりました。

In [ ]:
per_arm_py_env = p_a_env.StationaryStochasticPerArmPyEnvironment(
    global_context_sampling_fn,
    per_arm_context_sampling_fn,
    NUM_ACTIONS,
    linear_normal_reward_fn,
    batch_size=BATCH_SIZE
)
per_arm_tf_env = tf_py_environment.TFPyEnvironment(per_arm_py_env)

以下で、この環境が生成するものを確認できます。

In [ ]:
print('observation spec: ', per_arm_tf_env.observation_spec())
print('\nAn observation: ', per_arm_tf_env.reset().observation)

action = tf.zeros(BATCH_SIZE, dtype=tf.int32)
time_step = per_arm_tf_env.step(action)
print('\nRewards after taking an action: ', time_step.reward)

観測仕様は、以下の 2 つの要素を持つディクショナリであることがわかります。

- キー `'global'` を持つ要素: これはグローバルコンテキストの部分で、形状はパラメータ `GLOBAL_DIM` に一致します。
- キー `'per_arm'` を持つ要素: これがアーム単位のコンテキストで、その形状は `[NUM_ACTIONS, PER_ARM_DIM]` です。この部分は、実行ステップのすべてのアームのアーム特徴量のプレースホルダーです。


### LinUCB エージェント

LinUCB エージェントは、同盟のバンディットアルゴリズムを実行します。これは、線形報酬関数のパラメータを推定する一方で、その推定周囲の信頼楕円も維持します。エージェントは、パラメータがその信頼楕円内にあるものと仮定して、推定される最も高い期待報酬を持つアームを選択します。

エージェントを作成するには、観測とアクション仕様に関する知識が必要です。エージェントを定義する場合、ブール型パラメータ `accepts_per_arm_features` を `True` に設定します。

In [ ]:
observation_spec = per_arm_tf_env.observation_spec()
time_step_spec = ts.time_step_spec(observation_spec)
action_spec = tensor_spec.BoundedTensorSpec(
    dtype=tf.int32, shape=(), minimum=0, maximum=NUM_ACTIONS - 1)

agent = lin_ucb_agent.LinearUCBAgent(time_step_spec=time_step_spec,
                                     action_spec=action_spec,
                                     accepts_per_arm_features=True)

### トレーニングデータのフロー

このセクションでは、アーム単位の特徴がポリシーからトレーニングにどのように移行するかについての仕組みを簡単に説明します。 自由に次のセクション（後悔指標を定義する）に進み、興味があれば後でこのセクションに戻ることもできます。

まず、エージェントのデータ仕様を確認しましょう。エージェントの `training_data_spec` 属性は、トレーニングデータに必要な要素とトレーニングデータを指定します。

In [ ]:
print('training data spec: ', agent.training_data_spec)

仕様の `observation` の部分を詳しく見てみると、アーム単位の特徴量が含まれないことがわかります！

In [ ]:
print('observation spec in training: ', agent.training_data_spec.observation)

アーム単位の特徴量はどうなったのでしょうか。この疑問に答えるには、まず、LinUCB エージェントがトレーニングする際に、**すべての**アームのアーム単位の特徴量は不要であり、**選択された**アームのみのアーム単位の特徴量が必要であることに注意しましょう。したがって、形状 `[BATCH_SIZE, NUM_ACTIONS, PER_ARM_DIM]` のテンソルをドロップするのが合理的です。特にアクション数が多い場合には非常に無駄であるためです。

とは言え、それでも選択されたアームのアーム単位の特徴量はどこかに存在しなければなりません！これを確認するために、LinUCB ポリシーが、選択されたアームの特徴量をトレーニングデータの `policy_info` フィールド内に格納していることを確認します。

In [ ]:
print('chosen arm features: ', agent.training_data_spec.policy_info.chosen_arm_features)

この形状から、`chosen_arm_features` には 1 つのアームの特徴量ベクトルのみが含まれており、それが選択されたアームであることがわかります。`policy_info` と `chosen_arm_features` は、トレーニングデータの仕様を調べたところからわかるように、トレーニングデータの一部であるため、トレーニング時に利用できることに注意してください。

### 後悔指標を定義する

トレーニングループを開始する前に、エージェントの後悔の計算を支援するユーティリティ関数を定義します。これらの関数は、アクション（アームの特徴量によって指定される）とエージェントが表示できない線形パラメータのセットを基に、最適な期待報酬を判定する上で役立ちます。

In [ ]:
def _all_rewards(observation, hidden_param):
  """Outputs rewards for all actions, given an observation."""
  hidden_param = tf.cast(hidden_param, dtype=tf.float32)
  global_obs = observation['global']
  per_arm_obs = observation['per_arm']
  num_actions = tf.shape(per_arm_obs)[1]
  tiled_global = tf.tile(
      tf.expand_dims(global_obs, axis=1), [1, num_actions, 1])
  concatenated = tf.concat([tiled_global, per_arm_obs], axis=-1)
  rewards = tf.linalg.matvec(concatenated, hidden_param)
  return rewards

def optimal_reward(observation):
  """Outputs the maximum expected reward for every element in the batch."""
  return tf.reduce_max(_all_rewards(observation, reward_param), axis=1)

regret_metric = tf_bandit_metrics.RegretMetric(optimal_reward)

これで、バンディットトレーニングループを開始する準備が整いました。以下のドライバーはポリシーを使ってアクションの選択を処理し、選択されたアクションの報酬を再生バッファに格納し、事前定義済みの後悔指標を計算し、エージェントのトレーニングステップを実行します。

In [ ]:
num_iterations = 20 # @param
steps_per_loop = 1 # @param

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.policy.trajectory_spec,
    batch_size=BATCH_SIZE,
    max_length=steps_per_loop)

observers = [replay_buffer.add_batch, regret_metric]

driver = dynamic_step_driver.DynamicStepDriver(
    env=per_arm_tf_env,
    policy=agent.collect_policy,
    num_steps=steps_per_loop * BATCH_SIZE,
    observers=observers)

regret_values = []

for _ in range(num_iterations):
  driver.run()
  loss_info = agent.train(replay_buffer.gather_all())
  replay_buffer.clear()
  regret_values.append(regret_metric.result())


では、結果を見てみましょう。すべてを正しく行ったのであれば、エージェントは線形報酬関数をうまく推定できるため、ポリシーは最適な報酬に近い期待報酬を持つアクションを選択できます。これは、上記で定義した後悔指標で示されており、この指標は 0 に向かって低下します。

In [ ]:
plt.plot(regret_values)
plt.title('Regret of LinUCB on the Linear per-arm environment')
plt.xlabel('Number of Iterations')
_ = plt.ylabel('Average Regret')

### 今後の内容

上記の例は、[ニューラル Epsilon-Greedy エージェント](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/neural_epsilon_greedy_agent.py)など、他のエージェントからも選択できるコードベースに[実装](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/examples/v2/train_eval_per_arm_stationary_linear.py)されています。